# Imports

In [1]:
import sqlite3
import pandas as pd

In [2]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

In [50]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

In [3]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')


# Load Vector Store

In [7]:
# Load FAISS index
vectorstore = FAISS.load_local(
    "faiss_index", 
    OpenAIEmbeddings(model="text-embedding-ada-002"), 
    allow_dangerous_deserialization=True
)

# Initialize the language model (chat model)
llm = ChatOpenAI(model="gpt-4o",
                max_tokens=None,
                timeout=None,
                max_retries=5)

# Setup RAG 

In [8]:
# Set up Retrieval Augmented Generation (RAG)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())

In [9]:
def run_chat_query(query):
    return qa_chain.run(query)

In [12]:
run_chat_query("What is the main topic of the conference [<10 words]")

'Python-focused programming and technology advancements.'

# Setup SQL Database

In [25]:
def run_sql_query(query):
    # Create a SQLite connection
    conn = sqlite3.connect('data/conference_data.db')
    try:
        result = pd.read_sql_query(query, conn)
        print("Running SQL Query:")
        print(query)
        print(result)  
        return result
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        conn.close()
    return None


In [9]:
query = '''
SELECT 
    m.name AS table_name, 
    p.name AS column_name, 
    p.type AS data_type
FROM 
    sqlite_master m 
JOIN 
    pragma_table_info(m.name) p 
WHERE 
    m.type = 'table'
ORDER BY 
    table_name, p.cid;


'''
run_sql_query(query)

Running SQL Query:

SELECT 
    m.name AS table_name, 
    p.name AS column_name, 
    p.type AS data_type
FROM 
    sqlite_master m 
JOIN 
    pragma_table_info(m.name) p 
WHERE 
    m.type = 'table'
ORDER BY 
    table_name, p.cid;



   table_name              column_name data_type
0      github               speaker_id   INTEGER
1      github               github_url      TEXT
2      github                 username      TEXT
3      github                     name      TEXT
4      github             public_repos   INTEGER
5      github              total_stars   INTEGER
6      github   most_starred_repo_name      TEXT
7      github  most_starred_repo_stars   INTEGER
8      github     last_commit_datetime      TEXT
9      github  hours_since_last_commit      REAL
10     github                followers   INTEGER
11     github                following   INTEGER
12     github               created_at      TEXT
13     github                  company      TEXT
14     github               

,table_name,column_name,data_type
0,github,speaker_id,INTEGER
1,github,github_url,TEXT
2,github,username,TEXT
3,github,name,TEXT
4,github,public_repos,INTEGER
5,github,total_stars,INTEGER
6,github,most_starred_repo_name,TEXT
7,github,most_starred_repo_stars,INTEGER
8,github,last_commit_datetime,TEXT
9,github,hours_since_last_commit,REAL


In [13]:
query = '''
SELECT COUNT(*)
FROM talks t;
'''
run_sql_query(query)

Running SQL Query:

SELECT COUNT(*)
FROM talks t;

   COUNT(*)
0        36


,COUNT(*)
0,36


# Write Prompt route query type either to the LLM or to the SQL engine

In [14]:
question = "How many talks in each category?"

In [39]:
def get_prompt_decide_LLM_or_SQL(question):
    return f'''
    You are an expert analyst who can decide whether it is better to rely on a database of documents and general reasoning or whether we should use a sql query to answer a question
        
    Your job is to decide whether the input QUESTION would be better answered with SQL or LLM.
    
    A question answered better with SQL will tend to be analytical or questions about numbers.
    
    A question better answered with an LLM that has documents in its context via RAG would be something like "what is alex conway's talk about"
    
    The QUESTION is:
    {question}
    
    When you believe it is better to answer with the LLM and the document store, reply "LLM" and nothing else.
    
    When you believe it is better to write a SQL query and check the database to answer the question then reply "SQL" and nothing else.
        
        IMPORTANT instructions:
        1) Only reply with a single word - either SQL or LLM
    
    
    Here is the table structure of the sql data:
    github table:
    speaker_id,github_url,username,name,public_repos,total_stars,most_starred_repo_name,most_starred_repo_stars,last_commit_datetime,hours_since_last_commit,followers,following,created_at,company,bio,location,blog,email
    8,https://github.com/sixfeetup,sixfeetup,Six Feet Up,127,210,scaf,79,2024-09-25T17:04:43+00:00,164.0354158066667,19,0,2011-09-08T20:39:11+00:00,,"Accelerating IMPACT with App Dev, AI & Big Data","Fishers, IN",http://www.sixfeetup.com,info@sixfeetup.com
    20,https://github.com/gijzelaerr,gijzelaerr,Gijs Molenaar,172,748,python-snap7,648,2024-09-17T16:20:59+00:00,356.8318311875,188,111,2010-07-08T11:22:39+00:00,spotify,"astro software composer, music brewer, beer programmer, senior system engineer @spotify",Amsterdam - Windhoek,http://pythonic.nl,gijsmolenaar@gmail.com
    21,https://github.com/luisdza,luisdza,Luis de Sousa,8,3,aerialmzansi-website,2,2024-08-22T13:24:29+00:00,983.7772850216667,23,112,2014-04-12T05:41:28+00:00,@Syeop,Creating and dreaming,"Johannesburg, South Africa",,
    ...
    
    speakers table:
    Speaker ID,Name,Profile URL,Photo URL,Twitter,GitHub,Fediverse,Bio Part 1,Bio Part 2,Bio Part 3
    1,Alex Conway,https://2024.za.pycon.org/users/alxcnwy/,No photo available,https://twitter.com/alxcnwy,https://github.com/alxcnwy,,Twitter Profile link:https://twitter.com/alxcnwy,Other Social:https://www.linkedin.com/in/alxcnwy/,github profile:https://github.com/alxcnwy
    2,Nunudzai Mrewa,https://2024.za.pycon.org/users/JustNunuz/,No photo available,,https://github.com/JustNunuz,,Twitter Profile link:https://x.com/_absolute_z3r0,Other Social:https://www.linkedin.com/in/nunuz/,github profile:https://github.com/JustNunuz
    3,Brenden Taylor,https://2024.za.pycon.org/users/brenden.taylor22/,No photo available,,,,"I am a data scientist at Praelexis, in Stellenbosch. I have mostly worked in financial services, specifically in time series forecasting in Python.",,
    ...
    
    talks table:
    Talk ID,Title,Abstract,Speaker Name,Speaker Profile URL,Talk URL,Speaker ID
    1,Applying AI with Python,"Artificial Intelligence (AI) is transforming industries and our lives by enabling computers to solve increasingly complex tasks, and Python stands at the forefront of this revolution. In this talk, we will explore how Python's rich ecosystem of AI libraries and frameworks can be harnessed to quickly solve real-world problems without needing a PhD. From computer vision to time series analysis to natural language processing and AI agents, this session will demonstrate practical AI applications in real-world use cases that leverage Python's versatility so you can get started using AI right away.
    ...
    
    topics table:
    Talk ID,Assigned Topic
    1,AI and Machine Learning
    2,Community and Tech Education
    3,Time Series Analysis
    4,AI and Machine Learning
    5,Natural Language Processing (NLP)
    6,Community and Tech Education
    7,Community and Tech Education
    8,Community and Tech Education
    ...
    
    
    '''

In [40]:
prompt_decide_LLM_or_SQL = get_prompt_decide_LLM_or_SQL(question)
run_chat_query(prompt_decide_LLM_or_SQL)

'SQL'

In [42]:
def get_prompt_write_SQL_query_for_question(question):
    return f'''
    You are a Senior SQL Programmer who can understand the complex table structures and form SQL queries as per the user's question.
        
    Your job is to write a SQL query that answers the question and return the SQL code for the query and nothing else.
    
    The QUESTION is:
    {question}
    
        
        IMPORTANT instructions:
        1) Dont use JOIN if it is not necessary, check the table and its usable columns, if using JOIN do it with extra CAUTION
        2) Dont forget to apply inverted commas when required.
        3) Give no explanation and directly provide answers.
        4) Take care when joining tables, use right set of table-column configuration
        5) Return only a SQL query and nothing else
    
    Here is the table structure of the sql data:
    
       table_name              column_name data_type
    0      github               speaker_id   INTEGER
    1      github               github_url      TEXT
    2      github                 username      TEXT
    3      github                     name      TEXT
    4      github             public_repos   INTEGER
    5      github              total_stars   INTEGER
    6      github   most_starred_repo_name      TEXT
    7      github  most_starred_repo_stars   INTEGER
    8      github     last_commit_datetime      TEXT
    9      github  hours_since_last_commit      REAL
    10     github                followers   INTEGER
    11     github                following   INTEGER
    12     github               created_at      TEXT
    13     github                  company      TEXT
    14     github                      bio      TEXT
    15     github                 location      TEXT
    16     github                     blog      TEXT
    17     github                    email      TEXT
    18   speakers               speaker_id   INTEGER
    19   speakers                     name      TEXT
    20   speakers              profile_url      TEXT
    21   speakers                photo_url      TEXT
    22   speakers                  twitter      TEXT
    23   speakers                   github      TEXT
    24   speakers                fediverse      TEXT
    25   speakers               bio_part_1      TEXT
    26   speakers               bio_part_2      TEXT
    27   speakers               bio_part_3      TEXT
    28      talks                  talk_id   INTEGER
    29      talks                    title      TEXT
    30      talks                 abstract      TEXT
    31      talks             speaker_name      TEXT
    32      talks      speaker_profile_url      TEXT
    33      talks                 talk_url      TEXT
    34      talks               speaker_id   INTEGER
    35     topics                  talk_id   INTEGER
    36     topics           assigned_topic      TEXT
    
    
    
    Here is some smaple rows from the database:
    
    github table:
    speaker_id,github_url,username,name,public_repos,total_stars,most_starred_repo_name,most_starred_repo_stars,last_commit_datetime,hours_since_last_commit,followers,following,created_at,company,bio,location,blog,email
    8,https://github.com/sixfeetup,sixfeetup,Six Feet Up,127,210,scaf,79,2024-09-25T17:04:43+00:00,164.0354158066667,19,0,2011-09-08T20:39:11+00:00,,"Accelerating IMPACT with App Dev, AI & Big Data","Fishers, IN",http://www.sixfeetup.com,info@sixfeetup.com
    20,https://github.com/gijzelaerr,gijzelaerr,Gijs Molenaar,172,748,python-snap7,648,2024-09-17T16:20:59+00:00,356.8318311875,188,111,2010-07-08T11:22:39+00:00,spotify,"astro software composer, music brewer, beer programmer, senior system engineer @spotify",Amsterdam - Windhoek,http://pythonic.nl,gijsmolenaar@gmail.com
    21,https://github.com/luisdza,luisdza,Luis de Sousa,8,3,aerialmzansi-website,2,2024-08-22T13:24:29+00:00,983.7772850216667,23,112,2014-04-12T05:41:28+00:00,@Syeop,Creating and dreaming,"Johannesburg, South Africa",,
    ...
    
    speakers table:
    Speaker ID,Name,Profile URL,Photo URL,Twitter,GitHub,Fediverse,Bio Part 1,Bio Part 2,Bio Part 3
    1,Alex Conway,https://2024.za.pycon.org/users/alxcnwy/,No photo available,https://twitter.com/alxcnwy,https://github.com/alxcnwy,,Twitter Profile link:https://twitter.com/alxcnwy,Other Social:https://www.linkedin.com/in/alxcnwy/,github profile:https://github.com/alxcnwy
    2,Nunudzai Mrewa,https://2024.za.pycon.org/users/JustNunuz/,No photo available,,https://github.com/JustNunuz,,Twitter Profile link:https://x.com/_absolute_z3r0,Other Social:https://www.linkedin.com/in/nunuz/,github profile:https://github.com/JustNunuz
    3,Brenden Taylor,https://2024.za.pycon.org/users/brenden.taylor22/,No photo available,,,,"I am a data scientist at Praelexis, in Stellenbosch. I have mostly worked in financial services, specifically in time series forecasting in Python.",,
    ...
    
    talks table:
    Talk ID,Title,Abstract,Speaker Name,Speaker Profile URL,Talk URL,Speaker ID
    1,Applying AI with Python,"Artificial Intelligence (AI) is transforming industries and our lives by enabling computers to solve increasingly complex tasks, and Python stands at the forefront of this revolution. In this talk, we will explore how Python's rich ecosystem of AI libraries and frameworks can be harnessed to quickly solve real-world problems without needing a PhD. From computer vision to time series analysis to natural language processing and AI agents, this session will demonstrate practical AI applications in real-world use cases that leverage Python's versatility so you can get started using AI right away.
    ...
    
    topics table:
    Talk ID,Assigned Topic
    1,AI and Machine Learning
    2,Community and Tech Education
    3,Time Series Analysis
    4,AI and Machine Learning
    5,Natural Language Processing (NLP)
    6,Community and Tech Education
    7,Community and Tech Education
    8,Community and Tech Education
    ...
    
    '''

In [43]:
prompt_write_SQL_query_for_question = get_prompt_write_SQL_query_for_question(question)

In [44]:
sql_query = run_chat_query(prompt_write_SQL_query_for_question)
print(sql_query)

```sql
SELECT assigned_topic, COUNT(*) AS talk_count
FROM topics
GROUP BY assigned_topic;
```


In [31]:
# scrub sqlquery
sql_query=sql_query.replace("```sql\n","").replace("\n``","").replace("\n"," ")
sql_query

' SELECT assigned_topic, COUNT(*) AS talk_count  FROM topics  GROUP BY assigned_topic; '

In [32]:
# query sql database with generated query
sql_results = run_sql_query(sql_query)

Running SQL Query:
 SELECT assigned_topic, COUNT(*) AS talk_count  FROM topics  GROUP BY assigned_topic; 
                          assigned_topic  talk_count
0                AI and Machine Learning           4
1           Community and Tech Education           8
2                           Data Quality           1
3      Data Science and Decision Engines           1
4              Event Streaming and Kafka           1
5                    IoT and MicroPython           1
6      Natural Language Processing (NLP)           2
7   Open Source and Software Development           4
8             Python Libraries and Tools           9
9                               Robotics           1
10                  Time Series Analysis           1
11          Web Development and Frontend           3


In [46]:
def get_prompt_analyze_sql(question, sql_results):
    return f'''
    You are an expert analyst who can answer questions using supporting data which was returned from a SQL query on a dataset about the conference.
        
    Your job is to answer the input QUESTION, using the SQL_RESULTS in your answer and identifying any insights or trends you can from the results of the SQL query.
    
    The QUESTION is:
    {question}
    
    The SQL_RESULTS are:
    {sql_results}
    
    Important instructions
    * do not reference the SQL database or use the term SQL unless it's from a part of the data itself
    * be concise and try to make points with short bullet points
    * return the sql resutlts dataset as part of your response
    
    '''

In [47]:
prompt_analyze_sql = get_prompt_analyze_sql(question, sql_results)

In [49]:
print_markdown(run_chat_query(prompt_analyze_sql))

Here are the counts of talks in each category:

- **AI and Machine Learning:** 4
- **Community and Tech Education:** 8
- **Data Quality:** 1
- **Data Science and Decision Engines:** 1
- **Event Streaming and Kafka:** 1
- **IoT and MicroPython:** 1
- **Natural Language Processing (NLP):** 2
- **Open Source and Software Development:** 4
- **Python Libraries and Tools:** 9
- **Robotics:** 1
- **Time Series Analysis:** 1
- **Web Development and Frontend:** 3

**Insights:**
- The category "Python Libraries and Tools" has the highest number of talks (9).
- "Community and Tech Education" also has a significant number of talks (8).
- Several categories have only one talk, indicating niche topics (e.g., Data Quality, Robotics).

**SQL Results:**
```plaintext
                              assigned_topic  talk_count
0                AI and Machine Learning           4
1           Community and Tech Education           8
2                           Data Quality           1
3      Data Science and Decision Engines           1
4              Event Streaming and Kafka           1
5                    IoT and MicroPython           1
6      Natural Language Processing (NLP)           2
7   Open Source and Software Development           4
8             Python Libraries and Tools           9
9                               Robotics           1
10                  Time Series Analysis           1
11          Web Development and Frontend           3
```

Else LLM query:

In [51]:
def get_prompt_analyze_llm(question):
    return f'''
    You are an expert analyst who can answer questions using supporting documents.
        
    Your job is to answer the input QUESTION, using the documents in your answer and identifying any insights.
    
    The QUESTION is:
    {question}
    
    Important instructions
    * be concise and try to make points with short bullet points
    * answer only from the context from the documents, do not make up answers that are not supported by data
    
    '''

In [55]:
prompt_analyze_llm = get_prompt_analyze_llm(question)

print_markdown(run_chat_query(prompt_analyze_llm))

Here are the talks categorized based on the provided context:

- **Data Analysis/Processing:**
  - Talk ID: 24: "Accelerate your pandas workload using FireDucks at zero manual effort" by Sourav Saha
  - Talk ID: 22: "Duck, Duck, Python: OLAP Data with DuckDB" by Luis de Sousa

- **Data Quality:**
  - Talk ID: 15: "Great Expectations About Data Quality" by Ariella Rink

- **Artificial Intelligence:**
  - Talk ID: 1: "Applying AI with Python" by Alex Conway

Summary:
- Data Analysis/Processing: 2 talks
- Data Quality: 1 talk
- Artificial Intelligence: 1 talk

fails as expected

# Write function to run pipeline

In [56]:
def agent(question = "How many talks in each category?", debug=False):
    # figure out if SQL or LLM question
    prompt_decide_LLM_or_SQL = get_prompt_decide_LLM_or_SQL(question)
    question_type = run_chat_query(prompt_decide_LLM_or_SQL)
    
    if question_type == "SQL":
        # write sql query
        prompt_write_SQL_query_for_question = get_prompt_write_SQL_query_for_question(question)
        sql_query = run_chat_query(prompt_write_SQL_query_for_question)
        if debug:
            print("SQL query type")
            print(sql_query)
        
        # scrub sqlquery
        sql_query=sql_query.replace("```sql\n","").replace("\n``","").replace("\n"," ")
        
        # query sql database with generated query
        sql_results = run_sql_query(sql_query)

        # construct answer with insights from the data
        prompt_analyze_sql = get_prompt_analyze_sql(question, sql_results)
        
        return print_markdown(run_chat_query(prompt_analyze_sql))
    else:
        if debug:
            print("LLM query type")
        prompt_analyze_llm = get_prompt_analyze_llm(question)
        return print_markdown(run_chat_query(prompt_analyze_llm))

In [57]:
agent("What is Alex Conway speaking about?")

Alex Conway is speaking about the following:

- **Title**: Applying AI with Python
- **Abstract**: 
  - AI's impact on industries and everyday life
  - Python's role in enabling AI applications
  - Practical AI applications in computer vision, time series analysis, natural language processing, and AI agents
  - Demonstrations of real-world use cases and examples
- **Key Takeaways**:
  - Solid conceptual foundation for applying AI
  - Knowledge of useful Python libraries and tools for AI
  - Emphasis on accessibility of AI without needing advanced math skills

For more details, visit the talk URL: [Applying AI with Python](https://2024.za.pycon.org/talks/11-applying-ai-with-python/)

In [59]:
agent("How many speakers have active github profiles?")

Error occurred: Execution failed on sql 'SELECT COUNT(*) FROM speakers WHERE github IS NOT NULL AND github != '';`': You can only execute one statement at a time.


- **Speakers with Active GitHub Profiles:**
  - **Luis de Sousa**: GitHub profile at https://github.com/luisdza
  - **Sourav Saha**: GitHub profile at https://github.com/qsourav
  - **Cory Zue**: GitHub profile at https://github.com/czue
  - **Alex Conway**: GitHub profile at https://github.com/alxcnwy

**Total Number of Speakers with Active GitHub Profiles: 4**